# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Install TensorFlow 1.13.1 GPU versions


In [0]:
!pip install tensorflow-gpu==1.13.1

## Check GPU allocation
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Git Clone the donkey repository

In [0]:
!git clone https://github.com/autorope/donkeycar.git donkey

## Install donkey car

In [0]:
!pip3 install -e donkey

## Create Project

In [0]:
!donkey createcar --path /content/mycar

## Prepare Data
### Copy the following code and run on pi


```bash
cd ~/mycar
tar -czf data.tar.gz data

```

This will create a data.tar.gz file under ~/mycar/data

## Upload Data

### Option 1 - Upload directly to colab

Upload directly to colab could be quite slow. See other options below for faster upload methods.

#### Copy the data.tar.gz to your local pc

Run this on your local pc if you are using linux/mac
```
sftp pi@raspberry.local
cd ~/mycar/data
get data.tar.gz
```

If you are on a windows, download sftp utility like [filezilla](https://filezilla-project.org/) or [putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html)

#### Upload to Colab

In [0]:
import os
from google.colab import files

if(os.path.exists("/content/data.tar.gz")):
   os.remove("/content/data.tar.gz")
   !rm -rf /content/mycar/data

uploaded = files.upload()

WORK_FOLDER = "/content/mycar/data/"
if(os.path.exists(WORK_FOLDER) == False):
  os.makedirs(WORK_FOLDER)

!mv /content/data.tar.gz /content/mycar
%cd /content/mycar/data/
!tar -xzf data.tar.gz

### Option 2 - Upload to Google Drive

Alternatively, you could first upload the data.tar.gz to Google Drive. We will then mount Google Drive from colab and copy the data from Drive directly. When you run the cell below, you will need to click the link and generate an authorization code to for colab to access your drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Suppose you upload the data.tar.gz to Google Drive/mycar/data.tar.gz, this is how you copy it from Google Drive to colab

In [0]:
%cd /content/mycar
!cp /content/drive/My\ Drive/mycar/data.tar.gz .

And untar it to the right place

In [0]:
!tar -xzf data.tar.gz

Check whether the data is there

In [0]:
!ls data

## Train your model

In [0]:
!python /content/mycar/manage.py train --type linear --model /content/mycar/models/mypilot.h5

Check if the model is generated



In [0]:
!ls models

In [0]:
from IPython.display import Image
Image('models/mypilot.h5_loss_acc_0.086745.png')

## Copy trained model back to Donkey Car (Pi)

1. Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.  You can use the file tab on the left hand side to find the model and right click to download it.

![alt text](https://raw.githubusercontent.com/robocarstore/donkey-car-training-on-google-colab/master/images/screen_colab_download_model.png)

2. Copy the file from your PC or Mac to the Raspberry Pi using Filezilla or scp command.

In [0]:
from google.colab import files

%cd /content/mycar/models
!ls -ahl
files.download('mypilot.h5')

Alternatively, you can copy the model back to Google Drive too

In [0]:
!cp /content/mycar/models/mypilot.h5 /content/drive/My\ Drive/mycar/models

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```
